# Setup

In [ ]:
!pip install -q tensorflow==2.4.1

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.python.framework import smart_cond

import numpy as np

In [ ]:
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print("Device:", tpu.master())
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# except:
#     strategy = tf.distribute.get_strategy()
# print("Number of replicas:", strategy.num_replicas_in_sync)

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.python.framework import smart_cond

import numpy as np

def pruno_random_channels_first(similarity, seed, inputs_flat, actual_batchsize, fmap_count, fmap_size):
    fmap_even = (fmap_count//2)*2
    flatshape = (-1, fmap_count, fmap_size)
    gam = tf.math.reduce_mean(inputs_flat, axis=-1, keepdims=True)
    live = tf.cast(inputs_flat[:, :, :] > gam[:, :, :], dtype='float32', name='live')
    indices = tf.constant(np.arange(fmap_count), dtype='int32')
    random_indices = tf.random.shuffle(indices, seed=seed)
    mult_list = []
    for fm in range(0, fmap_even, 2):
        mult_list.append(live[:, random_indices[fm], :] * live[:, random_indices[fm + 1], :])
    mult = tf.stack(mult_list, axis=1, name='stack1')
    percent = tf.math.reduce_sum(mult, axis=-1, keepdims=True, name='percent') / flatshape[2]
    mask = tf.cast(percent < similarity, dtype='float32', name='mask')
    mask_unstack = tf.unstack(mask, axis=1, name='unstack1')
    mask_list = []
    for i in range(fmap_even // 2):
        mask_list.append(mask_unstack[i])
        mask_list.append(mask_unstack[i])
    if fmap_count > fmap_even:
        mask_list.append(tf.ones_like(mask_unstack[0], dtype='float32'))
    dup_mask = tf.stack(mask_list, axis=1, name='stack2')
    tiled_indices_flat = tf.tile(random_indices, actual_batchsize)
    tiled_indices = tf.reshape(tiled_indices_flat, (-1, flatshape[1], 1))
    inverse_mask_flat = tf.gather(dup_mask, tiled_indices, batch_dims=1, axis=1)
    inverse_mask = tf.reshape(inverse_mask_flat, (-1, fmap_count, 1))
    return inputs_flat * inverse_mask

def pruno_random_channels_last(similarity, seed, inputs_flat, actual_batchsize, fmap_count, fmap_size):
    fmap_even = (fmap_count//2)*2
    flatshape = (-1, fmap_size, fmap_count)
    gam = tf.math.reduce_mean(inputs_flat, axis=1, keepdims=True)
    live = tf.cast(inputs_flat[:, :, :] > gam[:, :, :], dtype='float32')
    indices = tf.constant(np.arange(fmap_count), dtype='int32')
    random_indices = tf.random.shuffle(indices, seed=seed)
    mult_list = []
    for fm in range(0, fmap_even, 2):
        mult_list.append(live[:, :, random_indices[fm]] * live[:, :, random_indices[fm + 1]])
    mult = tf.stack(mult_list, axis=2)
    percent = tf.math.reduce_sum(mult, axis=1, keepdims=True) / flatshape[1]
    mask = tf.cast(percent < similarity, dtype='float32')
    mask_unstack = tf.unstack(mask, axis=2)
    mask_list = []
    for i in range(fmap_even // 2):
        mask_list.append(mask_unstack[i])
        mask_list.append(mask_unstack[i])
    if fmap_even < fmap_count:
        mask_list.append(tf.ones_like(mask_unstack[0], dtype='float32'))
    dup_mask = tf.stack(mask_list, axis=1)
    tiled_indices_flat = tf.tile(random_indices, actual_batchsize)
    tiled_indices = tf.reshape(tiled_indices_flat, (-1, 1, fmap_count))
    inverse_mask_flat = tf.gather(dup_mask, tiled_indices, batch_dims=1, axis=1)
    inverse_mask = tf.reshape(inverse_mask_flat, (-1, 1, fmap_count))
    return inputs_flat * inverse_mask

def pruno_random_channels_batchwise(similarity, seed, inputs_flat, actual_batchsize, fmap_count, fmap_size):
    fmap_even = (fmap_count//2)*2
    flatshape = (-1, fmap_size, fmap_count)
    gam = tf.math.reduce_mean(inputs_flat, axis=1, keepdims=True)
    live = tf.cast(inputs_flat[:, :, :] > gam[:, :, :], dtype='float32')
    indices = tf.constant(np.arange(fmap_count), dtype='int32')
    random_indices = tf.random.shuffle(indices, seed=seed)
    print('random_indices:', random_indices)
    mult_list = []
    for fm in range(0, fmap_even, 2):
        mult_list.append(live[:, :, random_indices[fm]] * live[:, :, random_indices[fm + 1]])
    mult = tf.stack(mult_list, axis=2)
    percent = tf.math.reduce_sum(mult, axis=1, keepdims=True) / flatshape[1]
    mask = tf.cast(percent < similarity, dtype='float32')
    print('mask:', mask)
    # return mask
    mean_mask = tf.cast(tf.math.reduce_mean(mask, axis=0) > 0.50001, dtype='float32')
    mean_mask = tf.tile(tf.squeeze(mean_mask), actual_batchsize)
    mean_mask = tf.reshape(mean_mask, (-1, 1, fmap_even // 2))
    print('mean_mask:', mean_mask)
    # return mean_mask
    mask_unstack = tf.unstack(mean_mask, axis=2)
    mask_list = []
    for i in range(fmap_even // 2):
        mask_list.append(mask_unstack[i])
        mask_list.append(mask_unstack[i])
    if fmap_even < fmap_count:
        mask_list.append(tf.ones_like(mask_unstack[0], dtype='float32'))
    dup_mask = tf.stack(mask_list, axis=1)
    print('dup_mask:', dup_mask)
    # return dup_mask
    tiled_indices_flat = tf.tile(random_indices, actual_batchsize)
    tiled_indices = tf.reshape(tiled_indices_flat, (-1, 1, fmap_count))
    print('tiled_indices:', tiled_indices)
    # return tiled_indices
    inverse_mask_flat = tf.gather(dup_mask, tiled_indices, batch_dims=1, axis=1)
    inverse_mask = tf.reshape(inverse_mask_flat, (-1, 1, fmap_count))
    return inputs_flat * inverse_mask

def pruno_random_channels_norm_batchwise(similarity, seed, inputs_flat, actual_batchsize, fmap_count, fmap_size, batchwise=True):
    fmap_even = (fmap_count//2)*2
    flatshape = (-1, fmap_size, fmap_count)
    indices = tf.constant(np.arange(fmap_count), dtype='int32')
    random_indices = tf.random.shuffle(indices, seed=seed)
    # print('random_indices:', random_indices)
    inputs_norm, inputs_norm_sqrt = tf.linalg.normalize(inputs_flat, axis=1)
    # print('inputs_norm:', inputs_norm)
    # return inputs_norm
    mult_list = []
    for fm in range(0, fmap_even, 2):
        mult_list.append(inputs_norm[:, :, random_indices[fm]] * inputs_norm[:, :, random_indices[fm + 1]])
    mult = tf.stack(mult_list, axis=2)

    gam = tf.math.reduce_mean(mult, axis=1, keepdims=True)
    # print('gam:', gam)
    # return gam
    live = tf.cast(mult[:, :, :] > gam, dtype='float32')
    # print('live:', live)
    # return live
    percent = tf.math.reduce_sum(live, axis=1, keepdims=True) / flatshape[1]
    # print('percent:', percent)
    # return percent
    mask = tf.cast(percent < similarity, dtype='float32')
    # print('mask:', mask)
    # return mask
    if batchwise:
        mask = tf.cast(tf.math.reduce_mean(mask, axis=0) > 0.50001, dtype='float32')
        # I have no idea why these two variations are necessary
        if fmap_count > 3:
            mask = tf.squeeze(mask)
        else:
            mask = tf.reshape(mask, (1,))
        mask = tf.tile(mask, actual_batchsize)
        mask = tf.reshape(mask, (-1, 1, fmap_even // 2))
        print('batchwise')
    else:
        print('not batchwise')
    mask_unstack = tf.unstack(mask, axis=2)
    mask_list = []
    for i in range(fmap_even // 2):
        mask_list.append(mask_unstack[i])
        mask_list.append(mask_unstack[i])
    if fmap_even < fmap_count:
        mask_list.append(tf.ones_like(mask_unstack[0], dtype='float32'))
    dup_mask = tf.stack(mask_list, axis=1)
    # print('dup_mask:', dup_mask)
    # return dup_mask
    tiled_indices_flat = tf.tile(random_indices, actual_batchsize)
    tiled_indices = tf.reshape(tiled_indices_flat, (-1, 1, fmap_count))
    # print('tiled_indices:', tiled_indices)
    # return tiled_indices
    inverse_mask_flat = tf.gather(dup_mask, tiled_indices, batch_dims=1, axis=1)
    inverse_mask = tf.reshape(inverse_mask_flat, (-1, 1, fmap_count))
    # print('inverse_mask:', inverse_mask)
    # return inverse_mask
    return inputs_flat * inverse_mask

class Pruno2DLSTM(tf.keras.layers.Layer):
    """Applies Pruning Dropout to the input.
    
    Conv2DLSTM output: (batch, time, row, col, fmaps)
    
    The Pruno2DLSTM layer compares randomly chosen pairs of feature maps, and sets
    both feature maps to zero when they are "too similar". Similarity is measured
    by counting the pixels in both feature maps that are greater than the mean 
    of each feature map.
    When using `model.fit`,
    `training` will be appropriately set to True automatically, and in other
    contexts, you can set the kwarg explicitly to True when calling the layer.
    (This is in contrast to setting `trainable=False` for a Pruno layer.
    `trainable` does not affect the layer's behavior, as Pruno does
    not have any variables/weights that can be frozen during training.)
    >>> layer = Pruno(.2, seed=0, input_shape=(2, 5, 2))
    >>> data = np.arange(20).reshape(2, 5, 2).astype(np.float32)
    >>> print(data)
    [[[ 0.  1.]
    [ 2.  3.]
    [ 4.  5.]
    [ 6.  7.]
    [ 8.  9.]]
  
   [[10. 11.]
    [12. 13.]
    [14. 15.]
    [16. 17.]
    [18. 19.]]]
    >>> outputs = layer(data, training=True)
    >>> print(outputs)
    tf.Tensor(
    [[ 0.    1.25]
     [ 2.5   3.75]
     [ 5.    6.25]
     [ 7.5   8.75]
     [10.    0.  ]], shape=(5, 2), dtype=float32)
    Arguments:
      rate: Float between 0 and 1. 1.0 - rate = percentage of matching values 
      which triggers a dropout event
      seed: A Python integer to use as random seed.
    Call arguments:
      inputs: Input tensor (of any rank).
      training: Python boolean indicating whether the layer should behave in
        training mode (adding dropout) or in inference mode (doing nothing).
    """
  
    def __init__(self, similarity, batchwise=True, norm=False, sequence=True, seed=None, training=False, **kwargs):
        super(Pruno2DLSTM, self).__init__(**kwargs)
        if similarity < 0.0 or similarity > 1.0:
            raise ValueError('similarity must be between 0.0 and 1.0: %s' % str(similarity))
        self.similarity = similarity
        self.seed = seed
        self.batchwise = batchwise
        self.norm = norm
        self.sequence = sequence
        self.supports_masking = True
        self.training = training
        print('__init__: similarity:', similarity)
  
    def build(self, input_shape):
        print('Pruno2DLSTM.build: input_shape:', input_shape)
        self.built = True
  
    def call(self, inputs, training=None):
        if training is None:
            # do not activate in untrainable section of trainable model
            if self.training or self.trainable:
                training = K.learning_phase()
            else:
                training = False
        
        def identity_inputs():
            return inputs
  
        # each timestep is a separate batch
        # transpose to make timesteps * fmaps the final dimension
        def dropped_inputs():
            shape = inputs.shape.as_list()
            print('Pruno2DLSTM.call: shape as_list:', shape)
            actual_batchsize = tf.shape(inputs)[0:1]
            timesteps = shape[1]
            print('actual_batchsize, timesteps:', actual_batchsize, timesteps)
            self.fmap_shape = [shape[2], shape[3]]
            self.fmap_count = shape[4]
            if sequence:
                pass
            else:
                transposed = tf.transpose(inputs, perm=(0,2,3,1,4))
                print('transposed:', transposed)
                flatshape = (-1, self.fmap_shape[0] * self.fmap_shape[1], timesteps * self.fmap_count)
                print('flatshape:', flatshape)
                inputs_flatmap = tf.reshape(transposed, flatshape)
            if self.norm:
                outputs_flat = pruno_random_channels_norm_batchwise(self.similarity, self.seed, inputs_flatmap, actual_batchsize, 
                                 flatshape[2], flatshape[1], batchwise=self.batchwise)
            elif self.batchwise:
                outputs_flat = pruno_random_channels_batchwise(self.similarity, self.seed, inputs_flatmap, actual_batchsize, 
                                 flatshape[2], flatshape[1])
            else:
                outputs_flat = pruno_random_channels_last(self.similarity, self.seed, inputs_flatmap, actual_batchsize, 
                                 flatshape[2], flatshape[1])
            if sequence:
                pass
            else:
                out_shape = (-1, self.fmap_shape[0], self.fmap_shape[1], timesteps, self.fmap_count)
                print('out_shape:', out_shape)
                out_reshaped = tf.reshape(outputs_flat, out_shape)
                print('output reshaped:', transposed)
                out_trans = tf.transpose(out_reshaped, perm=(0,3,1,2,4))
                print('output transposed:', out_trans)
                outputs = tf.reshape(out_trans, tf.shape(inputs))
            return outputs
  
        output = smart_cond.smart_cond(training, dropped_inputs,
                                            identity_inputs)
        return output
  
    # why?
    def _get_noise_shape(self, inputs):
        input_shape = array_ops.shape(inputs)
        noise_shape = (input_shape[0], 1, 1, 1, 1)
        return noise_shape

    def compute_output_shape(self, input_shape):
        return input_shape
  
    def get_config(self):
        config = {
            'similarity': self.similarity,
            'batchwise': self.batchwise,
            'norm': self.norm,
            'seed': self.seed
        }
        base_config = super(Pruno2DLSTM, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [ ]:
def get_model_LSTM(similarity, input_shape):
    input = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.ConvLSTM2D(11,1, return_sequences=True)(input)
    print('convlstm2d:', x)
    x = Pruno2DLSTM(similarity=0.46)(x) # , input_shape=input_shape, training=True)(x)
    x = tf.keras.layers.Conv3D(
            filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
        )(x),
    model = tf.keras.Model(input, x)
    model.compile(loss="binary_crossentropy", optimizer="adadelta")
    model.summary()
    return model


In [ ]:
# Example
in_data = np.arange(7*3*5*2).reshape((1, 7, 3, 5, 2)).astype(np.float32)
out_data = np.arange(7*3*5*1).reshape((1, 7, 3, 5, 1)).astype(np.float32)
#pruno = Pruno2DLSTM(similarity=0.46, input_shape=(7,3,5,2))

model = get_model_LSTM(0.65, (7, 3,5,2))
model.fit(in_data, out_data, epochs=1)
# model.predict(data)

convlstm2d: KerasTensor(type_spec=TensorSpec(shape=(None, 7, 3, 5, 11), dtype=tf.float32, name=None), name='conv_lst_m2d_9/transpose_1:0', description="created by layer 'conv_lst_m2d_9'")
__init__: similarity: 0.46
Pruno2DLSTM.build: input_shape: (None, 7, 3, 5, 11)
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 7, 3, 5, 2)]      0         
_________________________________________________________________
conv_lst_m2d_9 (ConvLSTM2D)  (None, 7, 3, 5, 11)       616       
_________________________________________________________________
pruno2dlstm_5 (Pruno2DLSTM)  (None, 7, 3, 5, 11)       0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 3, 5, 1)        298       
Total params: 914
Trainable params: 914
Non-trainable params: 0
__________________________________________________________

In [ ]:


def get_model(similarity, input_shape):
    pruno = Pruno2D(similarity, input_shape)#, seed=0)
    model = tf.keras.models.Sequential(pruno)
    model.compile()
    return model

In [ ]:
# Example
data = np.arange(30).reshape(1, 3, 5, 2).astype(np.float32)
pruno = Pruno2D(similarity=0.46, input_shape=(3,5,2))
model = tf.keras.models.Sequential(pruno,)
model.compile()
model.predict(data)

In [ ]:
# 3 5x3 feature maps
fmaps = 2
shape = (5, 4)
samples = 2
quant = shape[0]*shape[1]
zeros = np.zeros(((1,) + shape + (fmaps,)))
data = np.zeros(((samples,) + shape + (fmaps,)))
data1 = np.arange(quant).reshape(1, shape[0], shape[1], 1).astype(np.float32)/quant
data1 = data1 * 1.58
data[0, :, :, :] = data1 
data[1, :, :, :] = np.sin(data1)
# data[1, :, :, :] = data1 + 1
# data[2, :, :, :] = data1 + 2

# data[0, 1, :, :] = np.sin(data1)
# data[0, 1, :, 0] = zeros[0, 0, :, 0]
# data[0, 1, :, 0:2] = zeros[0, 0, :, 0:2]

print('data.shape:', data.shape)

    # same, sim (.466 -> 4.67)
# 1/3 0, (0.333 -> 0.334) 
pruno = Pruno2D(similarity=0.5, input_shape=(shape+(fmaps,)))
model = tf.keras.models.Sequential(pruno,)
model.compile()
model.predict(data)

In [ ]:
# 4 5x3 feature maps
samples = 1
shape = (5, 3)
fmaps = 2
quant = shape[0] * shape[1]
data = np.zeros(((samples,) + shape + (fmaps,))) + 0.1
data1 = np.arange(quant).reshape((1, shape[0], shape[1], 1)).astype(np.float32)/quant
# data1 = data1 * 1.58
for i in range(samples-2):
    data[i, :, :, :] = data1[:, :, :, :]
    data[i, 0, :, 1] = np.sin(data1[:, 0, :, 0])
    data[i, 0, :, 2] = 0
    # data[i, 0:2, :, 3] = 0

print('data.shape:', data.shape)

# with strategy.scope():
model = get_model(similarity=0.5, input_shape=(shape + (fmaps,)))

pred = model.predict(data)
print('pred.shape:', pred.shape)
print(pred)

In [ ]:
# 4 5x3 feature maps
samples = 6
shape = (5, 3)
fmaps = 4
quant = shape[0] * shape[1]
data = np.zeros(((samples,) + shape + (fmaps,))) + 0.1
data1 = np.arange(quant).reshape((1, shape[0], shape[1], 1)).astype(np.float32)/quant
# data1 = data1 * 1.58
for i in range(samples-1):
    data[i, :, :, :] = data1[:, :, :, :]
    data[i, 0, :, 1] = np.sin(data1[:, 0, :, 0])
    data[i, 0, :, 2] = 0
    data[i, 0:2, :, 3] = 0

print('data.shape:', data.shape)

# with strategy.scope():
model = get_model(similarity=0.4, input_shape=(shape + (fmaps,)))

pred = model.predict(data)
print('pred.shape:', pred.shape)
print(pred)

In [ ]:
# 3 5x3 feature maps
fmaps = 4
shape = (5, 3)
samples = 1
quant = shape[0]*shape[1]
zeros = np.zeros(((1,) + shape + (fmaps,)))
data = np.zeros(((samples,) + shape + (fmaps,)))
data1 = np.arange(quant).reshape(1, shape[0], shape[1], 1).astype(np.float32)/quant
data1 = data1 * 1.58
data[0, :, :, :] = data1 
# data[1, :, :, :] = data1 + 1
# data[2, :, :, :] = data1 + 2

for i in range(samples):
    data[i, :, :, :] = data1[:, :, :, :]
    data[i, :, :, 1] = np.sin(data1[0, :, :, 0])
    data[i, 0, :, 2] = 0
    data[i, 0:2, :, 3] = 0

print('data.shape:', data.shape)

    # same, sim (.466 -> 4.67)
# 1/3 0, (0.333 -> 0.334) 
pruno = Pruno2D(similarity=0.47, input_shape=(shape+(fmaps,)))
model = tf.keras.models.Sequential(pruno)
model.compile()
model.predict(data)

In [ ]:
!!pip install -q git+https://github.com/LanceNorskog/keras-wedge.git
from keras_wedge_dropout import wedge_direct_comparison, wedge_normalize
import numpy as np

In [ ]:
img1 = np.asarray([[1,2],[3,4]])
img2 = np.asarray([[5,6],[7,3]])

print('Feature Map #1')
print(img1)
print('Feature Map #2')
print(img2)
print()

correlation1, percentage1 = wedge_direct_comparison(img1, img2)
print('Correlation (count > mean): ')
print(correlation1)
print('  similarity score:', percentage1)
correlation2, percentage2 = wedge_normalize(img1, img2)
print('Correlation (normalized and multiplied): ')
print(correlation2)
print('  similarity score:', percentage2)


Feature Map #1
[[1 2]
 [3 4]]
Feature Map #2
[[5 6]
 [7 3]]

Correlation (count > mean): 
[[ True False]
 [ True False]]
  similarity score: 0.5
Correlation (normalized and multiplied): 
[[False False]
 [ True False]]
  similarity score: 0.25
